In [1]:
import pandas as pd
import re
import preprocessor as p
import string
import emoji
import unicodedata
from nltk.corpus import stopwords
import json

## Importing JSON data

In [2]:
# Reading tweets from JSON file and ading them to Pandas Dataframe
tweets = pd.read_json('saver_output.json')

In [3]:
tweets.head(10)

,id,date,hashtags,text
0,1234267154897395712,2020-03-01 23:59:55+00:00,#coronavirus #Coronavid19 #coronavirusveneto,Cazzarola ora tutto pi chiaro Altro che cinesi...
1,1234267147569897472,2020-03-01 23:59:54+00:00,#domenicalive #noneladurso #Coronavid19 #coron...,566 nuovi casi dallinizio di domenicalive none...
2,1234267146693365760,2020-03-01 23:59:54+00:00,#coronavirus #scuole #conte #campania,coronavirus scuole conte campania ULTIMISSIMA ...
3,1234267068062666752,2020-03-01 23:59:35+00:00,#HoaraBorselli #AdrianoPanzironi #nonelarena #...,Il commento di HoaraBorselli contro AdrianoPan...
4,1234267052292104192,2020-03-01 23:59:31+00:00,,MotoGP 2020 Coronavirus Cancellati i primi due...
5,1234266987032961024,2020-03-01 23:59:15+00:00,#amore,amore ai tempi del corona virus mia figlia mi ...
6,1234266840014217216,2020-03-01 23:58:40+00:00,#coronavirus,Ho messo un aggregato dei dati disponibili su ...
7,1234266778915741696,2020-03-01 23:58:26+00:00,,Ecco le istruzioni per creare il vaccino del c...
8,1234266691028176896,2020-03-01 23:58:05+00:00,,Coronavirus buone regole
9,1234266617917386752,2020-03-01 23:57:47+00:00,#Coronavirus #informazione #rassegnastampa #Ra...,VIDEO Il Coronavirus monopolizza l informazion...


## Cleaning

### Preprocessor and emoji

In [4]:
# Setting preprocesor options
p.set_options(p.OPT.URL,p.OPT.MENTION)

In [5]:
# Function for cleaning with preprocessor
# It removes URLs and mentions
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [6]:
# Demojize
# Replaces emojis with text
def demojize(row):
    text = row['text']
    text = emoji.demojize(text)
    return text

In [7]:
# Applying previously defined functions
tweets['text'] = tweets.apply(preprocess_tweet, axis=1)
tweets['text'] = tweets.apply(demojize, axis=1)

In [8]:
tweets.head(10)

,id,date,hashtags,text
0,1234267154897395712,2020-03-01 23:59:55+00:00,#coronavirus #Coronavid19 #coronavirusveneto,Cazzarola ora tutto pi chiaro Altro che cinesi...
1,1234267147569897472,2020-03-01 23:59:54+00:00,#domenicalive #noneladurso #Coronavid19 #coron...,566 nuovi casi dallinizio di domenicalive none...
2,1234267146693365760,2020-03-01 23:59:54+00:00,#coronavirus #scuole #conte #campania,coronavirus scuole conte campania ULTIMISSIMA ...
3,1234267068062666752,2020-03-01 23:59:35+00:00,#HoaraBorselli #AdrianoPanzironi #nonelarena #...,Il commento di HoaraBorselli contro AdrianoPan...
4,1234267052292104192,2020-03-01 23:59:31+00:00,,MotoGP 2020 Coronavirus Cancellati i primi due...
5,1234266987032961024,2020-03-01 23:59:15+00:00,#amore,amore ai tempi del corona virus mia figlia mi ...
6,1234266840014217216,2020-03-01 23:58:40+00:00,#coronavirus,Ho messo un aggregato dei dati disponibili su ...
7,1234266778915741696,2020-03-01 23:58:26+00:00,,Ecco le istruzioni per creare il vaccino del c...
8,1234266691028176896,2020-03-01 23:58:05+00:00,,Coronavirus buone regole
9,1234266617917386752,2020-03-01 23:57:47+00:00,#Coronavirus #informazione #rassegnastampa #Ra...,VIDEO Il Coronavirus monopolizza l informazion...


### Re

In [9]:
# Deleting duplicate tweets
tweets = tweets.drop_duplicates(subset='text')
tweets['data'] = tweets['text']

In [10]:
# Defining stopwords and adding a space before and after to exclude the case
# in which the stopword is cointained in a word
words = set(stopwords.words('italian'))
stopwords = [' ' + x + ' ' for x in words]

In [11]:
# Removing the tags
tweets.text = tweets.text.replace(
    "@[\w]*[_-]*[\w]*", " ", regex=True)
# Removing the spaces in excess
tweets.text = tweets.text.replace('\s+', ' ', regex=True)
# Removing the space at the beginning
tweets.text = tweets.text.replace('^ ', '', regex=True)
# Removing the space at the end
tweets.text = tweets.text.replace(' $', '', regex=True)
# To lowercase
tweets.text = tweets.text.apply(
    lambda x: x.lower())
tweets.text = tweets.text.replace('^', ' ', regex=True)
tweets.text = tweets.text.replace('$', ' ', regex=True)

# Removing stopwords
for word in stopwords:
    tweets.text = tweets.text.replace(word, ' ', regex=True)

# Removing the space at the beginning and at the end
tweets.text = tweets.text.apply(lambda x: x.strip())
# Removing empty tweets
tweets = tweets[tweets.text != '']

In [12]:
tweets.head(10)

,id,date,hashtags,text,data
0,1234267154897395712,2020-03-01 23:59:55+00:00,#coronavirus #Coronavid19 #coronavirusveneto,cazzarola ora pi chiaro altro cinesi coronavir...,Cazzarola ora tutto pi chiaro Altro che cinesi...
1,1234267147569897472,2020-03-01 23:59:54+00:00,#domenicalive #noneladurso #Coronavid19 #coron...,566 nuovi casi dallinizio domenicalive nonelad...,566 nuovi casi dallinizio di domenicalive none...
2,1234267146693365760,2020-03-01 23:59:54+00:00,#coronavirus #scuole #conte #campania,coronavirus scuole conte campania ultimissima ...,coronavirus scuole conte campania ULTIMISSIMA ...
3,1234267068062666752,2020-03-01 23:59:35+00:00,#HoaraBorselli #AdrianoPanzironi #nonelarena #...,commento hoaraborselli adrianopanzironi nonela...,Il commento di HoaraBorselli contro AdrianoPan...
4,1234267052292104192,2020-03-01 23:59:31+00:00,,motogp 2020 coronavirus cancellati primi due g...,MotoGP 2020 Coronavirus Cancellati i primi due...
5,1234266987032961024,2020-03-01 23:59:15+00:00,#amore,amore tempi corona virus figlia aspettava irga...,amore ai tempi del corona virus mia figlia mi ...
6,1234266840014217216,2020-03-01 23:58:40+00:00,#coronavirus,messo aggregato dati disponibili coronavirus r...,Ho messo un aggregato dei dati disponibili su ...
7,1234266778915741696,2020-03-01 23:58:26+00:00,,ecco istruzioni creare vaccino coronavirus div...,Ecco le istruzioni per creare il vaccino del c...
8,1234266691028176896,2020-03-01 23:58:05+00:00,,coronavirus buone regole,Coronavirus buone regole
9,1234266617917386752,2020-03-01 23:57:47+00:00,#Coronavirus #informazione #rassegnastampa #Ra...,video coronavirus monopolizza informazione vuo...,VIDEO Il Coronavirus monopolizza l informazion...


### Further cleaning

In [13]:
# Removing usernames in case preprocessor didn't work
def remove_mentions(text):
    return re.sub(r"(?:\@|https?\://)\S+", "", text)


# Remove punctuation in case Re missed something
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [14]:
# Removing mentions
tweets['text'] = tweets['text'].apply(remove_mentions)

# Removing punctuation
tweets['text'] = tweets['text'].apply(remove_punctuations)

In [15]:
tweets.head(10)

,id,date,hashtags,text,data
0,1234267154897395712,2020-03-01 23:59:55+00:00,#coronavirus #Coronavid19 #coronavirusveneto,cazzarola ora pi chiaro altro cinesi coronavir...,Cazzarola ora tutto pi chiaro Altro che cinesi...
1,1234267147569897472,2020-03-01 23:59:54+00:00,#domenicalive #noneladurso #Coronavid19 #coron...,566 nuovi casi dallinizio domenicalive nonelad...,566 nuovi casi dallinizio di domenicalive none...
2,1234267146693365760,2020-03-01 23:59:54+00:00,#coronavirus #scuole #conte #campania,coronavirus scuole conte campania ultimissima ...,coronavirus scuole conte campania ULTIMISSIMA ...
3,1234267068062666752,2020-03-01 23:59:35+00:00,#HoaraBorselli #AdrianoPanzironi #nonelarena #...,commento hoaraborselli adrianopanzironi nonela...,Il commento di HoaraBorselli contro AdrianoPan...
4,1234267052292104192,2020-03-01 23:59:31+00:00,,motogp 2020 coronavirus cancellati primi due g...,MotoGP 2020 Coronavirus Cancellati i primi due...
5,1234266987032961024,2020-03-01 23:59:15+00:00,#amore,amore tempi corona virus figlia aspettava irga...,amore ai tempi del corona virus mia figlia mi ...
6,1234266840014217216,2020-03-01 23:58:40+00:00,#coronavirus,messo aggregato dati disponibili coronavirus r...,Ho messo un aggregato dei dati disponibili su ...
7,1234266778915741696,2020-03-01 23:58:26+00:00,,ecco istruzioni creare vaccino coronavirus div...,Ecco le istruzioni per creare il vaccino del c...
8,1234266691028176896,2020-03-01 23:58:05+00:00,,coronavirus buone regole,Coronavirus buone regole
9,1234266617917386752,2020-03-01 23:57:47+00:00,#Coronavirus #informazione #rassegnastampa #Ra...,video coronavirus monopolizza informazione vuo...,VIDEO Il Coronavirus monopolizza l informazion...


In [16]:
# Remove accents in order to mitigate weird charachters in the final result
def strip_accents(row):
    text = row['text']
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return text

In [17]:
# Removing accent
tweets['text'] = tweets.apply(strip_accents, axis=1)

In [18]:
tweets.head(10)

,id,date,hashtags,text,data
0,1234267154897395712,2020-03-01 23:59:55+00:00,#coronavirus #Coronavid19 #coronavirusveneto,cazzarola ora pi chiaro altro cinesi coronavir...,Cazzarola ora tutto pi chiaro Altro che cinesi...
1,1234267147569897472,2020-03-01 23:59:54+00:00,#domenicalive #noneladurso #Coronavid19 #coron...,566 nuovi casi dallinizio domenicalive nonelad...,566 nuovi casi dallinizio di domenicalive none...
2,1234267146693365760,2020-03-01 23:59:54+00:00,#coronavirus #scuole #conte #campania,coronavirus scuole conte campania ultimissima ...,coronavirus scuole conte campania ULTIMISSIMA ...
3,1234267068062666752,2020-03-01 23:59:35+00:00,#HoaraBorselli #AdrianoPanzironi #nonelarena #...,commento hoaraborselli adrianopanzironi nonela...,Il commento di HoaraBorselli contro AdrianoPan...
4,1234267052292104192,2020-03-01 23:59:31+00:00,,motogp 2020 coronavirus cancellati primi due g...,MotoGP 2020 Coronavirus Cancellati i primi due...
5,1234266987032961024,2020-03-01 23:59:15+00:00,#amore,amore tempi corona virus figlia aspettava irga...,amore ai tempi del corona virus mia figlia mi ...
6,1234266840014217216,2020-03-01 23:58:40+00:00,#coronavirus,messo aggregato dati disponibili coronavirus r...,Ho messo un aggregato dei dati disponibili su ...
7,1234266778915741696,2020-03-01 23:58:26+00:00,,ecco istruzioni creare vaccino coronavirus div...,Ecco le istruzioni per creare il vaccino del c...
8,1234266691028176896,2020-03-01 23:58:05+00:00,,coronavirus buone regole,Coronavirus buone regole
9,1234266617917386752,2020-03-01 23:57:47+00:00,#Coronavirus #informazione #rassegnastampa #Ra...,video coronavirus monopolizza informazione vuo...,VIDEO Il Coronavirus monopolizza l informazion...


## Exporting

In [21]:
# Saving to CSV
tweets.to_csv('dataCleaned.csv')